# Eventdata und Verhältnisberechnung für ROMYADR/RLAS

In [ ]:
import matplotlib
matplotlib.use("Agg")  

In [ ]:
import os
import sys
from obspy.taup import TauPyModel
import timeit
import gc
import glob
import warnings
from datetime import datetime
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.gridspec import GridSpec
from PIL import Image
import plotly.graph_objects as go
from scipy import fftpack
from scipy.signal import hilbert, savgol_filter
import obspy as obs
from obspy import read, UTCDateTime, Stream, read_inventory
from obspy.clients.fdsn import Client, RoutingClient
from obspy.geodetics import gps2dist_azimuth, locations2degrees
from obspy.signal import array_analysis as AA
from obspy.signal.util import util_geo_km
from functions.add_distances_and_backazimuth import __add_distances_and_backazimuth
from functions.querrySeismoData import __querrySeismoData
from functions.hilfsfunktionen import safe_divide
from functions.computeADRromy import (__compute_romy_adr,__get_data, __adjust_time_line, __check_samples_in_stream)
from andbro__fft import __fft

### Methods

In [ ]:
def makeplotadrrlas_oadr(st, config, ev_lat, ev_lon, backazimuth):
    st_in = st.select(component="Z")
    
    if len(st_in) == 0:
        print(" -> keine Z-Komponente gefunden.")
        return

    rot_scaling = 1e9
    rot_unit = "nm/s"
    font = 14

    # Initialisierung
    tr1, tr2, tr3 = None, None, None
    
    # Stationen gezielt zuordnen
    for tr in st_in:
        if tr.stats.station.upper() == "ROMY" and tr.stats.location == "22":
            tr1 = tr
            data1 = tr1.data * rot_scaling
            spec1, ff1, ph1 = __fft(data1 ,tr.stats.delta, window=None, normalize=None)
        
        elif tr.stats.station.upper() == "RLAS":
            tr2 = tr
            data2 = tr2.data * rot_scaling
            spec2, ff2, ph2 = __fft(data2, tr.stats.delta, window=None, normalize=None)
    
    # Sicherstellen, dass beide Stationen gefunden wurden
    if tr1 is None or tr2 is None:
        print(" -> Fehlende Station: FUR oder WET nicht im Stream.")
        return
    if ev_lat > 30 and ev_lat < 60 and ev_lon > -50 and ev_lon < 50:
        region = 1
        print("Region 1 (Nahbeben)")
    elif ev_lat > 15 and ev_lat < 60 and ev_lon > 60 and ev_lon < 180:
        region = 2
        print("Region 2 (Asien und Philippinen)")
    elif ev_lat > -10 and ev_lat < 15 and ev_lon > 100 and ev_lon < 180:
        region = 2
        print("Region 2 (Asien und Philippinen)")
    elif ev_lat > 5 and ev_lat < 60 and ev_lon > -170 and ev_lon < -60:
        region = 4
        print("Region 4 (Nord- und Südamerika)")
    elif ev_lat > -80 and ev_lat < 5 and ev_lon > -120 and ev_lon < -50:
        region = 4
        print("Region 4 (Nord- und Südamerika)")
    elif ev_lat > -60 and ev_lat < -10 and (ev_lon > 90 or ev_lon < -150):
        region = 6
        print("Region 6 (Ozeanien)")
    else:
        region = 7
        print("Region 7 (nicht zugeordnet)")
    
    envelope1 = np.abs(hilbert(data1))
    envelope2 = np.abs(hilbert(data2))
    spec1_s = pd.Series(spec1).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    spec2_s = pd.Series(spec2).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    spec1_s = savgol_filter(spec1_s, window_length=81, polyorder=3)
    spec2_s = savgol_filter(spec2_s, window_length=81, polyorder=3)
    lines = ["⌀ Spectral ratio:"]
    ratio_22 = safe_divide(spec1_s, spec2_s)  
    freq_bands = [(round(f, 5), round(f + 0.02, 5)) for f in np.arange(0.04, 0.18 + 0.001, 0.01)]

    # Mittelwerte und Mittenfrequenzen extrahieren
    band_centers = [(fmin + fmax)/2 for fmin, fmax in freq_bands]
    mean_ratios_22 = []
    if not isinstance(ratio_22, int):
        for fmin_r, fmax_r in freq_bands:
            mask = (ff1 >= fmin_r) & (ff1 <= fmax_r)
            if np.any(mask): 
                mean_ratios_22.append(np.mean(ratio_22[mask]))
            else:
                mean_ratios_22.append(np.nan)

     # Farben definieren
    colors = {
        'tr1': '#1f77b4',        # Blau
        'envelope1': '#2ca02c',  # Grün
        'tr2': '#ff7f0e',        # Orange
        'envelope2': '#17becf',  # Türkis
        'spec1': '#1f77b4',      # Blau
        'spec2': '#ff7f0e',      # Orange
    }   
    unit = rot_unit
    fig, axes = plt.subplots(5,figsize=(18,5*4), sharex=False)
    plt.subplots_adjust(hspace=0.3)

    axes[0].plot(tr1.times(), data1, label=f"{tr1.stats.station} {tr1.stats.location} {tr1.stats.channel}", color=colors['tr1'], lw=1.0)
    axes[0].plot(tr1.times(), envelope1, color=colors['envelope1'], label='Envelope ' + tr1.stats.channel) 
    axes[0].set_ylabel(f'envelope and v ({unit})', fontsize=font)
    axes[0].legend()
    axes[0].text(0.02, 0.95, "a)", transform=axes[0].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    axes[1].plot(tr2.times(), data2, label=f"{tr2.stats.station} {tr2.stats.channel}", color=colors['tr2'], lw=1.0)
    axes[1].plot(tr2.times(), envelope2, color=colors['envelope2'], label='Envelope ' + tr2.stats.channel)
    axes[1].set_ylabel(f'Envelope and v ({unit})', fontsize=font)
    axes[1].set_xlabel(f"Time from {tr.stats.starttime.date} {str(tr.stats.starttime.time)[:8]} (s)", fontsize=font)
    axes[1].set_xlabel(f"Time from {tr.stats.starttime.date} {str(tr.stats.starttime.time)[:8]} (s)",fontsize=font)  
    axes[1].legend()
    axes[1].text(0.02, 0.95, "b)", transform=axes[1].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    axes[2].plot(ff1, spec1, color=colors['spec1'], lw=1.0, label=f"Spectra {tr1.stats.station} {tr1.stats.location} {tr1.stats.channel}")
    axes[2].plot(ff2, spec2, color=colors['spec2'], lw=1.0, label=f"Spectra {tr2.stats.station} {tr2.stats.channel}")
    axes[2].set_ylabel(f'ASD \n({unit}/Hz)', fontsize=font)
    axes[2].set_xlabel(f"Frequency (Hz)",fontsize=font) 
    axes[2].legend()
    axes[2].text(0.02, 0.95, "c)", transform=axes[2].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    axes[3].plot(ff1, spec1_s, color=colors['spec1'], lw=1.0, 
                 label=f"Spectra (smoothed) {tr1.stats.station} {tr1.stats.location} {tr1.stats.channel}")
    axes[3].plot(ff2, spec2_s, color=colors['spec2'], lw=1.0, 
                 label=f"Spectra (smoothed) {tr2.stats.station} {tr2.stats.channel}")
    axes[3].set_ylabel(f'ASD \n({unit}/Hz)', fontsize=font)
    axes[3].legend()
    axes[3].text(0.02, 0.95, "d)", transform=axes[3].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    axes[4].plot(band_centers, mean_ratios_22, marker='o', linestyle='None', color='black',
                 label=f"⌀ Spectral ratio {tr1.stats.station} {tr1.stats.location}/{tr2.stats.station}")
    axes[4].set_ylabel(f"⌀ Spectral ratio", fontsize=font)
    axes[4].grid(True, which='both', linestyle='None', alpha=0.5)
    axes[4].legend()
    axes[4].set_ylim(0,3)
    axes[4].text(0.02, 0.95, "e)", transform=axes[4].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    
    if "fmin" in config.keys() and "fmax" in config.keys():
        axes[2].set_xlim(config['fmin'],config['fmax'])
        axes[3].set_xlim(config['fmin'],config['fmax'])
        axes[4].set_xlim(config['fmin'],config['fmax'])
       
        
    fig.suptitle(f"M{magnitude:.1f} - {distance_km:.0f} km @ {ev_depth:.0f} km | {origin_time} UTC", fontsize=font+2)

    result_dict = {
            "origin_time": str(origin_time),
            "region": region,
            "magnitude": magnitude,
            "distance": distance_km,
            "depth": ev_depth,
            "lat" : ev_lat,
            "lon": ev_lon,
            "backazimuth": backazimuth
            }
    if not isinstance(ratio_22, int):
        for i, center in enumerate(band_centers):
            result_dict[f"ratio_{center:.4f}Hz"] = mean_ratios_22[i]
    else:
        for i, center in enumerate(band_centers):
            result_dict[f"ratio_{center:.3f}Hz"] = mean_ratio

    del st_in
    return fig, result_dict

In [ ]:
def makeplotadrrlas_aadr(st, config, ev_lat, ev_lon, backazimuth):
    st_in = st.select(component="Z")
    if len(st_in) == 0:
        print(" -> keine Z-Komponente gefunden.")
        return

    rot_scaling = 1e9
    rot_unit = "nm/s"
    font = 14

    # Initialisierung
    tr1, tr2, tr3 = None, None, None
    
    # Stationen gezielt zuordnen
    for tr in st_in:
        if tr.stats.station.upper() == "ROMY" and tr.stats.location == "23":
            tr3 = tr
            data3 = tr3.data * rot_scaling
            spec3, ff3, ph3 = __fft(data3 ,tr.stats.delta, window=None, normalize=None)
        elif tr.stats.station.upper() == "RLAS":
            tr2 = tr
            data2 = tr2.data * rot_scaling
            spec2, ff2, ph2 = __fft(data2, tr.stats.delta, window=None, normalize=None)
    
    # Sicherstellen, dass beide Stationen gefunden wurden
    if tr3 is None or tr2 is None:
        print(" -> Fehlende Station: FUR oder WET nicht im Stream.")
        return
    if ev_lat > 30 and ev_lat < 60 and ev_lon > -50 and ev_lon < 50:
        region = 1
        print("Region 1 (Nahbeben)")
    elif ev_lat > 15 and ev_lat < 60 and ev_lon > -60 and ev_lon < 180:
        region = 2
        print("Region 2 (Asien und Philippinen)")
    elif ev_lat > -10 and ev_lat < 15 and ev_lon > 100 and ev_lon < 180:
        region = 2
        print("Region 2 (Asien und Philippinen)")
    elif ev_lat > 5 and ev_lat < 60 and ev_lon > -150 and ev_lon < -60:
        region = 4
        print("Region 4 (Nord- und Südamerika)")
    elif ev_lat > -80 and ev_lat < 5 and ev_lon > -120 and ev_lon < -50:
        region = 4
        print("Region 4 (Nord- und Südamerika)")
    elif ev_lat > -60 and ev_lat < -10 and (ev_lon > 90 or ev_lon < -150):
        region = 6
        print("Region 6 (Ozeanien)")
    else:
        region = 7
        print("Region 7 (nicht zugeordnet)")
    
    envelope3 = np.abs(hilbert(data3))
    envelope2 = np.abs(hilbert(data2))
    spec2_s = pd.Series(spec2).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    spec3_s = pd.Series(spec3).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
    spec2_s = savgol_filter(spec2_s, window_length=61, polyorder=3)
    spec3_s = savgol_filter(spec3_s, window_length=61, polyorder=3)
    lines = ["⌀ Spectral ratio:"] 
    ratio_23 = safe_divide(spec3_s, spec2_s)
    freq_bands = [(round(f, 5), round(f + 0.02, 5)) for f in np.arange(0.04, 0.18 + 0.001, 0.01)]

    # Mittelwerte und Mittenfrequenzen extrahieren
    band_centers = [(fmin + fmax)/2 for fmin, fmax in freq_bands]
    mean_ratios_23 = []
    if not isinstance(ratio_23, int):
        for fmin_r, fmax_r in freq_bands:
            mask = (ff3 >= fmin_r) & (ff3 <= fmax_r)
            if np.any(mask): 
                mean_ratios_23.append(np.mean(ratio_23[mask]))
            else:
                mean_ratios_23.append(np.nan)

     # Farben definieren
    colors = {
        'tr1': '#1f77b4',        # Blau
        'envelope1': '#2ca02c',  # Grün
        'tr2': '#ff7f0e',        # Orange
        'envelope2': '#17becf',  # Türkis
        'spec1': '#1f77b4',      # Blau
        'spec2': '#ff7f0e',      # Orange
        'spec3': '#9467bd'       #lila
    }   
    unit = rot_unit
    fig, axes = plt.subplots(5,figsize=(18,5*4), sharex=False)
    plt.subplots_adjust(hspace=0.3)

    axes[0].plot(tr3.times(), data3, label=f"{tr3.stats.station} {tr3.stats.location} {tr3.stats.channel}", color=colors['tr1'], lw=1.0)
    axes[0].plot(tr3.times(), envelope3, color=colors['envelope1'], label='Envelope ' + tr3.stats.channel) 
    axes[0].set_ylabel(f'envelope and v ({unit})', fontsize=font)
    axes[0].legend()
    axes[0].text(0.02, 0.95, "a)", transform=axes[0].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    
    axes[1].plot(tr2.times(), data2, label=f"{tr2.stats.station} {tr2.stats.channel}", color=colors['tr2'], lw=1.0)
    axes[1].plot(tr2.times(), envelope2, color=colors['envelope2'], label='Envelope ' + tr2.stats.channel)
    axes[1].set_ylabel(f'Envelope and v ({unit})', fontsize=font)
    axes[1].set_xlabel(f"Time from {tr.stats.starttime.date} {str(tr.stats.starttime.time)[:8]} (s)", fontsize=font)
    axes[1].set_xlabel(f"Time from {tr.stats.starttime.date} {str(tr.stats.starttime.time)[:8]} (s)",fontsize=font)  
    axes[1].legend()
    axes[1].text(0.02, 0.95, "b)", transform=axes[1].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    
    axes[2].plot(ff3, spec3, color=colors['spec1'], lw=1.0, label=f"Spectra {tr3.stats.station} {tr3.stats.location} {tr3.stats.channel}")
    axes[2].plot(ff2, spec2, color=colors['spec2'], lw=1.0, label=f"Spectra {tr2.stats.station} {tr2.stats.channel}")
    axes[2].set_ylabel(f'ASD \n({unit}/Hz)', fontsize=font)
    axes[2].set_xlabel(f"Frequency (Hz)",fontsize=font) 
    axes[2].legend()
    axes[2].text(0.02, 0.95, "c)", transform=axes[2].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    
    axes[3].plot(ff3, spec3_s, color=colors['spec1'], lw=1.0, 
                 label=f"Spectra (smoothed) {tr3.stats.station} {tr3.stats.location} {tr3.stats.channel}")
    axes[3].plot(ff2, spec2_s, color=colors['spec2'], lw=1.0, 
                 label=f"Spectra (smoothed) {tr2.stats.station} {tr2.stats.channel}")
    axes[3].set_ylabel(f'ASD \n({unit}/Hz)', fontsize=font)
    axes[3].legend()
    axes[3].text(0.02, 0.95, "d)", transform=axes[3].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    
    axes[4].plot(band_centers, mean_ratios_23, marker='o', linestyle='None', color='black',
                 label=f"⌀ Spectral ratio {tr3.stats.station} {tr3.stats.location}/{tr2.stats.station}")
    axes[4].set_ylabel(f"⌀ Spectral ratio", fontsize=font)
    axes[4].grid(True, which='both', linestyle='None', alpha=0.5)
    axes[4].legend()
    axes[4].set_ylim(0,3)
    axes[4].text(0.02, 0.95, "e)", transform=axes[4].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    
    if "fmin" in config.keys() and "fmax" in config.keys():
        axes[2].set_xlim(config['fmin'],config['fmax'])
        axes[3].set_xlim(config['fmin'],config['fmax'])
        axes[4].set_xlim(config['fmin'],config['fmax'])
       
        
    fig.suptitle(f"M{magnitude:.1f} - {distance_km:.0f} km @ {ev_depth:.0f} km | {origin_time} UTC", fontsize=font+2)

    result_dict = {
            "origin_time": str(origin_time),
            "region": region,
            "magnitude": magnitude,
            "distance": distance_km,
            "depth": ev_depth,
            "lat" : ev_lat,
            "lon": ev_lon,
            "backazimuth": backazimuth
            }
    if not isinstance(ratio_23, int):
        for i, center in enumerate(band_centers):
            result_dict[f"ratio_{center:.4f}Hz"] = mean_ratios_23[i]
    else:
        for i, center in enumerate(band_centers):
            result_dict[f"ratio_{center:.3f}Hz"] = mean_ratio

    del st_in
    return fig, result_dict

### Configurations

In [ ]:
config = {}

# ROMY coordinates
config['ROMY_lon'] = 11.275501
config['ROMY_lat'] = 48.162941

# duration of event in seconds
config['duration'] = 7200

# frequency range for bandpass filter
config['fmin'] = 0.04 # in Hz   urspr. 0.01
config['fmax'] = 0.2 # in Hz    urspr. 0.1

# path for figures to store
config['outpath_figs'] = "C:/Bachelorarbeit/figures/geschwindigkeit/ADRRLAS/"

# path for output data
config['outpath_data'] = "C:/Bachelorarbeit/data/waveformsADRRLAS/"

# specify seed codes of stations that should be used for the analysis
config['seeds'] = [#"BW.ROMY.10.BJZ", "BW.ROMY..BJU", "BW.ROMY..BJV", "BW.ROMY..BJW", # ringlaser ROMY
                   "BW.RLAS..BJZ", # ringlaser G
                  ]

# path to catalogs
config['path_to_catalog'] = "C:/Bachelorarbeit/data/catalogs/"

# catalog file
config['catalog'] = "ROMY_global_catalog_20190101_20250531.pkl"
# set reference station
config['reference']= "GR.FUR"
    # specify stations to exclude
config['exclude'] = ['BW.ALFT']
    # ROMY coordinates
config['sta_lon'] = 11.275501
config['sta_lat'] = 48.162941
    # set verbose
config['verbose'] = True

## Load Events

In [ ]:
events = pd.read_pickle(config['path_to_catalog']+config['catalog'])

In [ ]:
events['origin'] = events.timestamp

In [ ]:
# make sure only events with magnitude > 6 are considered
events = events[events.magnitude > 6]
print("Event number: ", events.shape[0])

In [ ]:
# avoid events that are too close to each other in time
events['elapsed_time'] = events.timestamp.diff()
events = events[events.elapsed_time > pd.Timedelta(minutes=60)]
print("Event number: ", events.shape[0])

# RUN LOOP

Loop over events in catalog and request data for each event. Store data or event plots if
- skip existing files
- save figures
- store waveforms

is set in configurations, respectively.



In [ ]:
errors = []
model = TauPyModel(model="prem")
results_oadr = []
results_aadr = []
i = 0
mode = "good"
#für nur ADR/RLAS
skip_indices = {0,1,3,5,6,11,13,14,15,18,19,20,22,23,27,32,33,
                36,38,40,42,43,44,46,47,51,52,53,56,57,58,59,60,
               61,62,64,65,66,67,68,69,70,71,72,73,74,24,26,28,45,30,35,50,4,2}
#für ROMYADR/RLAS
#good_indices = {3,13,26,80,81,84,95,97,107,149,150,151,152,184,186,297,304,313,318,333,331,337,338,339,346}
good_indices = {9,13,23,31,34,35,38,40,42,55,70,93,98,110,148,151,173,216,217,
                220,221,353,356,363,372,397,405,219,255,387,397,396,405}
duration = {
#   i: duration 
    9: 3000, 23: 2000,31: 700,34: 1300,35: 950,38: 1300,40: 1400,55: 1700,70: 550,74: 2100,80: 400,93: 1000,98: 1000,110: 450, 148: 1400, 
    151: 1500, 173: 1000, 216: 1000,217: 1000, 220: 1000, 221: 2500, 353: 1500, 356: 1500, 363: 1500}
start ={
    23: 2000,31: 1800,34: 1650,35: 1850,38: 2500,53: 2000,40: 2600,55: 1800,70: 450,80: 300,93: 1200,98: 2500,110: 200, 148: 1600, 
    151: 2500, 173: 1800,216: 1200, 217: 2500, 220: 2000,221: 1500,353: 1500,356: 1500, 363: 500}
# loop over all events
for jj in range(events.shape[0]):
    num = str(jj).rjust(3, "0")
    print(f"\n -> {num} {events.origin.iloc[jj]}")
    if mode == "skip":
        if i in skip_indices:   
            print("skipped", i)
            i += 1
            continue
    if mode == "good":
        if not i in good_indices:      
            print("skipped", i)
            i += 1
            continue
    i += 1
    # create event name
    try:
        event_name = str(events.origin.iloc[jj]).replace("-", "").replace(":", "").replace(" ", "_").split(".")[0]
    except:
        print(f" -> {num}: error for {events.origin.iloc[jj]}")
        continue

    event_name_oadr = f"{event_name}_oadr.mseed"
    event_name_aadr = f"{event_name}_aadr.mseed"
      # event metadata
    ev_lat = events.latitude.iloc[jj]
    ev_lon = events.longitude.iloc[jj]
    sta_lat = config['ROMY_lat']
    sta_lon = config['ROMY_lon']
    distance_m, az, backazimuth = gps2dist_azimuth(ev_lat, ev_lon, sta_lat, sta_lon)
    distance_km = distance_m / 1000
    ev_depth = events.depth.iloc[jj] / 1000
    origin_time = obs.UTCDateTime(events.origin.iloc[jj])
    magnitude = events.magnitude.iloc[jj]
    distance_deg = locations2degrees(ev_lat, ev_lon, sta_lat, sta_lon)
     # set config
    config['title'] = f"M{magnitude:.1f} - {distance_km:.0f} km @ {ev_depth:.0f} km | {origin_time} UTC"
    starttime=origin_time
    config['tbeg'] = starttime + start.get(jj, 0)
    arrivals = model.get_travel_times(source_depth_in_km=ev_depth,
                                  distance_in_degree=distance_deg)
    
    if len(arrivals) == 0:
        print(f"Keine P-Ankunft für Event {jj} (Tiefe {ev_depth} km, Distanz {distance_deg}°)")
        starttime = origin_time
    else:
        p_arrival_time = arrivals[0].time 
        print(p_arrival_time)
        starttime = origin_time + p_arrival_time - 20 
    config['duration'] = duration.get(jj)
    
    if config['duration'] is None:
        if distance_km < 2000:
            config['duration'] = 1000
        elif 2000 < distance_km < 5000:
            config['duration'] = 1500
        elif 5000 < distance_km < 8000:
            config['duration'] = 2500
        elif 8000 < distance_km < 10000:
            config['duration'] = 3000
        elif 10000 < distance_km < 14000:
            config['duration'] = 4000
        elif 14000 < distance_km < 15000:
            config['duration'] = 4500

    config['tend'] = config['tbeg'] + config['duration']


    st0 = obs.Stream()
    if os.path.isfile(config['outpath_data']+f"{num}_{event_name}.mseed"):
        st0 = read(config['outpath_data']+f"{num}_{event_name}.mseed")
        data_vorhanden = True
    else:
        data_vorhanden = False
        st0 = obs.Stream()
        for seed in config['seeds']:
            try:                   
                if "RLAS" in seed:
                    stx, invx = __querrySeismoData(
                        seed_id=seed,
                        starttime=config['tbeg'],
                        endtime=config['tend'],
                        repository='george',
                        path=None,
                        restitute=True,
                        detail=None,
                        fill_value=None,
                        )
                else:
                    print(f" -> {seed} not found")
                    continue
    
                if len(stx) == 0:
                    print(f" -> data missing for {seed}")

                st0 += stx

            except Exception as e:
                print(e)
                print(f" -> failed to request {seed} for event: {origin_time}")
                errors.append(f" -> failed to request {seed} for event: {origin_time}")
                continue
        try:
            
            if not data_vorhanden:
                oadr = __compute_romy_adr(config['tbeg'],
                              config['tend'],
                              submask='outer',
                              ref_station=config['reference'],
                              verbose=config['verbose'],
                              excluded_stations=config['exclude'],
                              map_plot=False,
                             )

            oadr = oadr.trim(config['tbeg'], config['tend'], nearest_sample=False)
        
        except Exception as e:
            print(e)
            continue      
        # try to store data

        try:
            if not data_vorhanden:
                oadr.write(config['outpath_data']+event_name_oadr)
                print(f"stored under {config['outpath_data']+event_name_oadr}")
        except Exception as e:
            print(e)
        print(f"oadr result: {oadr}")

        #Compute ADR for entire array
        try:
            if not data_vorhanden:
                aadr = __compute_romy_adr(config['tbeg'],
                                      config['tend'],
                                      submask='all',
                                      ref_station=config['reference'],
                                      verbose=config['verbose'],
                                      excluded_stations=config['exclude'],
                                      map_plot=False,
                                     )
    
                aadr = aadr.trim(config['tbeg'], config['tend'], nearest_sample=False)

        except Exception as e:
                print(e)
                continue
         
    
        # try to store data
        try:
                if not data_vorhanden:
                    aadr.write(config['outpath_data']+event_name_aadr)
                    print(f"stored under {config['outpath_data']+event_name_aadr}")
        except Exception as e: 
                print(e)
        print(f"aadr result: {aadr}")
    if not data_vorhanden:
        st0 += oadr
        st0 += aadr
    # sort stream
    st0 = st0.sort()

    # fill masked data
    for tr in st0:
        if isinstance(tr.data, np.ma.MaskedArray):
            print(f" -> {tr.stats.channel} has masked data. Filled with zeros.")
            tr.data = tr.data.filled(fill_value=0)

    # preprocess
    print(" -> processing data stream ...")
    st1 = st0.copy()
    st1 = st1.detrend("linear")
    st1 = st1.taper(0.1)
    st1 = st1.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=4, zerophase=True)

    st1_rtz = st1

    # trim streams
    st1_rtz = st1_rtz.trim(config['tbeg'], config['tend'])
    st0 = st0.trim(config['tbeg'], config['tend'])
    if len(st1_rtz) <1:
        continue
    if not data_vorhanden:
        waveform_filename = f"{num}_{event_name}.mseed"
        if not os.path.isdir(config['outpath_data']):
            print("created: ", config['outpath_data'])
            os.makedirs(config['outpath_data'])

        try:
            st0.write(config['outpath_data'] + waveform_filename)
            print(f" -> stored at: {config['outpath_data'] + waveform_filename}")
        except Exception as e:
            print(f" -> error storing waveform: {e}")
            errors.append(f" -> error storing waveform: {e}")
            
 
    fig7, results_oadr = makeplotadrrlas_oadr(st1_rtz, config, ev_lat, ev_lon, backazimuth)
    fig8, results_aadr = makeplotadrrlas_aadr(st1_rtz, config, ev_lat, ev_lon, backazimuth)
    df_oadr = pd.DataFrame([results_oadr])
    df_aadr = pd.DataFrame([results_aadr])
    oadr_result_path = config['outpath_figs'] + f"oadr_results/{num}_{event_name}.csv"
    aadr_result_path = config['outpath_figs'] + f"aadr_results/{num}_{event_name}.csv"
    df_oadr.to_csv(oadr_result_path, index=False)
    df_aadr.to_csv(aadr_result_path, index=False)
    filtered_path = config['outpath_figs']+f"eventplots/"
    fig7.savefig(filtered_path + f"{num}_{event_name}_Spektrum_envelope_oadr-smooth.png", dpi=150, bbox_inches='tight', pad_inches=0.05)
    fig8.savefig(filtered_path + f"{num}_{event_name}_Spektrum_envelope_aadr-smooth.png", dpi=150, bbox_inches='tight', pad_inches=0.05)
    fig7.clf()
    plt.close(fig7)
    plt.close(fig8)
    del fig7, df_oadr, df_aadr, results_oadr, results_aadr
    st0 = None
    st1 = None
    st1_rtz = None
    gc.collect()
# show errors
pprint(errors)


In [ ]:
# oadr
oadr_files = glob.glob(config['outpath_figs'] + "oadr_results/*.csv")
df_all_oadr = pd.concat([pd.read_csv(f) for f in oadr_files], ignore_index=True)
df_all_oadr.to_csv(config['outpath_figs'] + "spektralanalyse_ergebnisse_oadr.csv", index=False)
# aadr
aadr_files = glob.glob(config['outpath_figs'] + "aadr_results/*.csv")
df_all_aadr = pd.concat([pd.read_csv(f) for f in aadr_files], ignore_index=True)
df_all_aadr.to_csv(config['outpath_figs'] + "spektralanalyse_ergebnisse_aadr.csv", index=False)